In [112]:
import wget
import tarfile
from os import listdir, getcwd
from os.path import isfile, join
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
import keras
import re
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, LSTM, embeddings

pd.set_option("display.width", 150)

First, let's go get the data. The IMDB reviews are hosted on Stanford's servers. This little block does a couple things:
  1. Pull the tarball down from Stanford's site. I'm on Windows, so use the wget package. On a Linux machine, you can use subprocess.call() to run wget from the shell.
  2. Unzip the tarball, and place it in the current directory (or you can specify one).

In [11]:
f = wget.download('https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz') # can use out= to place it somewhere
tar = tarfile.open(f, "r:gz")
tar.extractall() # path= can change directory
tar.close()

100% [....................................................] 84125825 / 84125825

..* a
..* b

Next, we'll parse the unzipped tructure. Like many ML datasets, the positive/negative sentiment and train/test are split out by nested directories. For explortaion, we're going to put it all in a pandas DF.

The folder structure goes like this:

```
aclImdb/
│
└───train/  
│   └─── pos/  
│   └─── neg/  
└───test/  
│   └─── pos/  
│   └─── neg/  
``` 

Within there are files for the review URLs, the vocabulary of unique words, and an extra folder of uncategorized reviews.

Each file is named file_<id>_<rating>.txt, so we're going to initialize a dataframe, and go through each folder to append the reviews. From the folders, we get:
  1. train/test split
  2. response (positive or negative sentiment)
  3. from the filename we get the actual rating and
  4. from the file we get the review text
  
*reword this*

In [2]:
df = pd.DataFrame(columns = ['split','rating','sentiment','review'])

for split in ['train', 'test']:
    for sent in ['pos', 'neg']:
        loc = join('.', 'aclImdb', split, sent) # Change the '.' with the path, if aclImdb is in a different folder
        for file in [f for f in listdir(loc) if isfile(join(loc, f)) and f.split('.')[-1]=='txt']:
            with open(join(loc,file),'r', encoding="utf-8") as f:
                text = f.read()
            df = df.append({'split':split,
                            'rating': file.split('_')[-1].split('.')[0],
                            'sentiment':sent,
                            'review':text}, ignore_index=True)

In [127]:
df.to_pickle('df_saved')

In [113]:
df.head()

,split,rating,sentiment,review
0,train,9,pos,"Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as ""Teachers"". My 35 years in the teaching profession lead me to believe that Bromwell Hig..."
1,train,8,pos,Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to sc...
2,train,10,pos,"Brilliant over-acting by Lesley Ann Warren. Best dramatic hobo lady I have ever seen, and love scenes in clothes warehouse are second to none. The corn on face is a classic, as good as anything in..."
3,train,7,pos,"This is easily the most underrated film inn the Brooks cannon. Sure, its flawed. It does not give a realistic view of homelessness (unlike, say, how Citizen Kane gave a realistic view of lounge si..."
4,train,8,pos,"This is not the typical Mel Brooks film. It was much less slapstick than most of his movies and actually had a plot that was followable. Leslie Ann Warren made the movie, she is such a fantastic, ..."


A few times throughout, we're going to need to flatten the review column into a blob of text. To ease this, the below helper function will be employed. It emplays two sets of ```" ".join()```s with an argument-less ```str.split()``` in the middle. The first combines the ```series.tolist()``` output, then the split breaks by spaces, tabs and linebreaks, and the outer join brings it all into a blob.

In [124]:
def flatten(ser):
    if not isinstance(ser, pd.Series):
        raise ValueError("Please pass a series")
    return " ".join(" ".join(ser.tolist()).split())

Using a counter, let's look at how many unique words there are

In [125]:
from collections import Counter

cnt = Counter(flatten(df.review).split())
print("There are {} unique tokens. The top are: \n\n{}".format(len(cnt),cnt.most_common(40)))

There are 469106 unique tokens. The top are: 

[('the', 568729), ('a', 306944), ('and', 301910), ('of', 283603), ('to', 261837), ('is', 203043), ('in', 169978), ('I', 133366), ('that', 126814), ('this', 113731), ('it', 107919), ('was', 92655), ('as', 83126), ('with', 82567), ('for', 80917), ('The', 68905), ('but', 66278), ('on', 61191), ('movie', 60761), ('are', 56507), ('film', 54276), ('his', 54200), ('have', 54007), ('not', 52140), ('you', 50700), ('be', 50537), ('at', 42420), ('by', 42289), ('he', 42115), ('one', 41334), ('an', 41076), ('from', 37878), ('who', 37207), ('like', 36028), ('all', 35279), ('they', 35201), ('has', 32419), ('so', 32387), ('just', 32337), ('or', 32217)]


Dealing with natural text is, frankly, a pain. 

As you can already see here, things like capitalization (this vs This) are going to be an issue. But so will punctuation, since reviewers don't use it consistently:

In [115]:
print("The word can't appears {} times, and cant {} times.".format(cnt["can't"],cnt["cant"]))

The word can't appears 6698 times, and cant 426 times.


To train our word vectors, we're going to want to clean this up a bit.

---
## Cleaning and Tokenizing

To help reduce the volume of our text, we're going to employ regular expressions to stripe out the non-alphanumeric characters. So can't -> cant will reduce our text down. After making that function, we'll tokenize the sentences and words, then run the regex to remove crud.

This isn't a perfect method and, frankly, there *may* be times where you want to capture all unique tokens entered by users. However, like all things in statistics, it's hard to estimate the effect of sparse parameters (and text is sparse enough as it is!).

First things first, did you notice ('/><br') above? This is because the text contains ```<br />``` HTML breaks. We're going to just outright remove them with some regular expressions (regex) to save a lot of headache later.

In [88]:
df["review"] = df["review"].apply(lambda x: re.sub('<.*?br.*?>', '', x))

With that done, let's set up our text cleaning function, stripping out the non-alphanumeric characters and making everything lower cased.

In [90]:
def simple_clean(text):
    return re.sub(r'([^\s\w]|_)+', '', text.lower())

simple_clean("While it's a little like doing surgery with a hacksaw, this sort of cleaning-method is quick and effective")

'while its a little like doing surgery with a hacksaw this sort of cleaningmethod is quick and effective'

Just how much might this reduce our text volume?

In [122]:
print("Reduced to {} unique tokens".format(len(Counter(flatten(df.review.apply(simple_clean)).split()))))

Reduced to 221858 unique tokens


52% reduction in tokens just from that!

This will help. But before we actually do the regex on our reviews, we're going to tokenize the sentences. This will be relevant when we start embedding with word2vec. NLTK includes a sentence tokenizer from the punkt collection.

Sentence tokenization isn't extremely straighforward--we can split on punctuation marks (.?!) but would need some regex to handle honorifics (Dr., Mr., D.M.D) and then random abbreviations would wreak havoc too (approx., lt., loc.).

Instead, we can use a sentence tokenizer that's already been trained on English. The one NLTK includes, punkt, uses an unsupervised method to learn sentences.

*If you've never used nltk's collections before, you'll need to download them by calling ```nltk.download()``` in a python shell. If you're unable to download the collections, don't worry. We can just split the text on (.?!) and it will be good enough.*

In [76]:
txt = """This is the first sentence.
Lt. Dan told me that this was the second.
Tokenizing sentences is an approx. art, but Dr. Somebody D.M.D says otherwise."""

print(sent_tokenize(txt))

['This is the first sentence.', 'Lt. Dan told me that this was the second.', 'Tokenizing sentences is an approx.', 'art, but Dr. Somebody D.M.D says otherwise.']


As you can see, it's not perfect. "approx." seems like it wasn't learned, but everything else tokenized correctly. When we send our text to word2vec, we're going to have it train on context windows around words. However, we want these windows to only include words from that sentence, since the next sentence could be totally unrelated.

This puts us with a dilema: Do we use the tokenizer, and know that some sentences will get fragmented, or just split on (.?!) knowing some context will leak between sentences?

We're going to do the latter, but I've used the former before with trememndous success, so it's not a bad option.

In [126]:
clean_reviews = [simple_clean(txt) for txt in sent_tokenize(flatten(df.review))]

print(clean_reviews[0:10])

['bromwell high is a cartoon comedy', 'it ran at the same time as some other programs about school life such as teachers', 'my 35 years in the teaching profession lead me to believe that bromwell highs satire is much closer to reality than is teachers', 'the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students', 'when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled  at  high', 'a classic line inspector im here to sack one of your teachers', 'student welcome to bromwell high', 'i expect that many adults of my age think that bromwell high is far fetched', 'what a pity that it isnt', 'homelessness or houselessness as george carlin stated has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school work or v